# Train with time-300B 

We'll  training the two architectures (LSTM and Neural ODE) using time-300B taking windows of 7, 14 and 21 sequenses.

This will be the experiments that we are going to do for both architectures:

- Baseline $100 \%$ of dataset

- Baseline $10 \%$ of dataset

- Baseline $3 \%$ of dataset

- Destillation with Time-Moe $3 \%$ of dataset

In [1]:
# Cargar la data
# Seleccionar las ventanas de cada sequencia de serie
# Dividir el porcentaje con muestreos (Separar 80-20)
# Entrenar los modelos
# Hacer destilación


### Load data

In [1]:
import random
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from Time_MoE.time_moe.datasets.time_moe_dataset import TimeMoEDataset
from sklearn.model_selection import train_test_split
# Importing custom functions
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)

from baseline.functions import load_data,create_intervals,create_windows,smape,smape_chunked,sample_fraction
from sklearn.metrics import mean_absolute_error, mean_squared_error

def load_data_clean():
    ds = TimeMoEDataset(data_folder='Time-300B\healthcare',normalization_method='zero')

    verbose = True
    total = len(ds)
    valid_indices = []
    # Iterar y filtrar
    for i in range(total):
        try:
            seq = ds[i]  # seq es numpy.ndarray según comprobaste
        except Exception as e:
            # Si hay error al obtener la secuencia, lo avisamos y saltamos
            if verbose:
                print(f"Advertencia: no se pudo obtener ds[{i}]: {e}")
            continue
        
        # Comprobación: si todos los valores son NaN, lo descartamos
        # seq es numpy.ndarray; cuidado si dims especiales, pero np.isnan funcionará elementwise.
        try:
            if not np.all(np.isnan(seq)):
                valid_indices.append(i)
        except Exception as e:
            # En caso de que seq no sea array puro, convertir primero:
            try:
                arr = np.array(seq)
                if not np.all(np.isnan(arr)):
                    valid_indices.append(i)
            except Exception as e2:
                if verbose:
                    print(f"Error al verificar NaN en secuencia índice {i}: {e2}")
                # Decidir si incluirla o no. Aquí optamos por descartarla:
                continue
    
    valid_count = len(valid_indices)
    if verbose:
        print(f"Secuencias totales en ds: {total}")
        print(f"Secuencias válidas (no todo NaN): {valid_count}")
        print(f"Secuencias descartadas: {total - valid_count}")
        sequences_validas = []

    for idx in valid_indices:
        try:
            sequences_validas.append(ds[idx])
        except Exception as e:
            if verbose:
                print(f"Error al extraer ds[{idx}] después de filtrar: {e}")
            # Podrías decidir saltar o detener. Aquí solo saltamos.
    return sequences_validas

def create_windows_from_sequences(sequences, window_size=15, horizon=1):
    """
    Dada una lista de secuencias (numpy arrays 1D), crea ventanas deslizantes:
    - X: array de shape (num_samples, window_size, 1)
    - y: array de shape (num_samples,)
    Cada muestra usa window_size pasos para predecir el siguiente valor (horizon=1).
    """
    X_list = []
    y_list = []
    for seq in sequences:
        # Asegurar numpy array
        arr = np.array(seq).astype(float)
        T = arr.shape[0]
        # Solo si la longitud es mayor que window_size + horizon - 1
        if T >= window_size + horizon:
            for start in range(0, T - window_size - horizon + 1):
                window = arr[start:start+window_size]
                target = arr[start+window_size:start+window_size+horizon]
                # Para horizon=1, target es un array de longitud 1; tomamos el escalar
                X_list.append(window.reshape(window_size, 1))
                y_list.append(target[0] if horizon == 1 else target)
    if len(X_list) == 0:
        return np.empty((0, window_size, 1)), np.empty((0,))
    X = np.stack(X_list, axis=0)
    y = np.array(y_list)

    # Supongamos X tiene forma (N, window_size, 1), y y forma (N,)
    mask_valid = ~np.isnan(X).any(axis=(1,2)) & ~np.isnan(y)
    # Mantener solo muestras sin NaN:
    X_clean = X[mask_valid]
    y_clean = y[mask_valid]
    print("De", X.shape[0], "muestras, quedan", X_clean.shape[0], "sin NaN")

    return X_clean, y_clean

def build_lstm_model(window_size=15, n_features=1, lstm_units=50):
    model = Sequential([
        LSTM(lstm_units, input_shape=(window_size, n_features)),
        Dense(1)  # para predicción de un valor escalar siguiente
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [2]:
ds = load_data_clean()

X, y = create_windows_from_sequences(ds, window_size=15, horizon=1)

Secuencias totales en ds: 1752
Secuencias válidas (no todo NaN): 1752
Secuencias descartadas: 0
De 433317 muestras, quedan 433317 sin NaN


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

### LSTM - 100 %

In [ ]:
# Suponiendo que X_train, y_train, X_val, y_val están listos:
model = build_lstm_model(window_size=15, n_features=1, lstm_units=50)
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[es]
)


KeyboardInterrupt: 

In [8]:
# Evaluar en conjunto de validación

y_pred = model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
#smape_value = smape_new(y_val, y_pred)

print("MAE:", mae)
print("MSE:", mse)
#print("SMAPE:", smape_value)

2709/2709 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
MAE: 0.2671545822468988
MSE: 0.24234376171207261


In [9]:
smape_val = smape_chunked(y_val, y_pred, chunk_size=500_000)
print("SMAPE:", smape_val)

SMAPE: 55.25035085214161


In [ ]:
# Guardar el modelo entrenado
model.save('Models_lstm/lstm_healthcare_model_100.keras')

### LSTM - 10 %



In [ ]:
# function to take only a percentage of the data
X_10, y_10 = sample_fraction(X, y, 0.10, random_state=42)

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_10, y_10, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
# Suponiendo que X_train, y_train, X_val, y_val están listos:
model = build_lstm_model(window_size=15, n_features=1, lstm_units=50)
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[es]
)

Epoch 1/20


c:\Users\criju\.conda\envs\ts\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1084/1084 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.3657 - val_loss: 0.2746
Epoch 2/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.2964 - val_loss: 0.2593
Epoch 3/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.2817 - val_loss: 0.2501
Epoch 4/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2538 - val_loss: 0.2524
Epoch 5/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.2723 - val_loss: 0.2501
Epoch 6/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.2532 - val_loss: 0.2478
Epoch 7/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.2554 - val_loss: 0.2491
Epoch 8/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.2509 - val_loss: 0.2480
Epoch 9/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.2599 - val_loss: 0.2465
Epoch 10/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.2604 - val_loss: 0.2463
Epoch 11/20
1084/1084 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.2602 - val_loss: 0.2498
Epoch 12/20
1084/1084 ━━━━━━━━━━━━━━━━━━━

In [16]:
# Evaluar en conjunto de validación

y_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
smape_val = smape_chunked(y_val, y_pred, chunk_size=500_000)

print("MAE:", mae)
print("MSE:", mse)
print("SMAPE:", smape_val)

271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
MAE: 0.28106132670530165
MSE: 0.24634181455920554
SMAPE: 59.27840290397485


In [17]:
# Guardar el modelo entrenado
model.save('Models_lstm/lstm_healthcare_model_10.keras')

### LSTM - 3 %

In [18]:
X_3, y_3   = sample_fraction(X, y, 0.03, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_3, y_3, test_size=0.2, shuffle=True, random_state=42)

In [19]:
# Suponiendo que X_train, y_train, X_val, y_val están listos:
model = build_lstm_model(window_size=15, n_features=1, lstm_units=50)
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[es]
)

Epoch 1/20


c:\Users\criju\.conda\envs\ts\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


325/325 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4545 - val_loss: 0.3020
Epoch 2/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3038 - val_loss: 0.3009
Epoch 3/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2906 - val_loss: 0.2900
Epoch 4/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2798 - val_loss: 0.2832
Epoch 5/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2763 - val_loss: 0.2773
Epoch 6/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2563 - val_loss: 0.2736
Epoch 7/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2638 - val_loss: 0.2709
Epoch 8/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2461 - val_loss: 0.2654
Epoch 9/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2549 - val_loss: 0.2638
Epoch 10/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2619 - val_loss: 0.2634
Epoch 11/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2447 - val_loss: 0.2621
Epoch 12/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.

In [20]:
# Evaluar en conjunto de validación

y_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
smape_val = smape_chunked(y_val, y_pred, chunk_size=500_000)

print("MAE:", mae)
print("MSE:", mse)
print("SMAPE:", smape_val)

82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
MAE: 0.28537223200023826
MSE: 0.2600540885694726
SMAPE: 59.34711632361779


In [21]:
# Guardar el modelo entrenado
model.save('Models_lstm/lstm_healthcare_model_3.keras')

### LSTM Distillation 

In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import Loss
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# Load time_moe
import torch
X_3, y_3   = sample_fraction(X, y, 0.03, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_3, y_3, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
# Convertir las ventanas a tensores para el modelo Time-MoE
import torch
import numpy as np    

tensors_train = []
tensors_val = []
for arr in X_train:                   
    t = torch.from_numpy(arr.reshape(1, 15)).float()  
    tensors_train.append(t)

for arr in X_val:                   
    t = torch.from_numpy(arr.reshape(1, 15)).float()  
    tensors_val.append(t)

In [24]:
# Teacher prediction using Time-MoE
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    'Maple728/TimeMoE-50M',
    device_map="cpu",  # use "cpu" for CPU inference, and "cuda" for GPU inference.
    trust_remote_code=True,
)

# forecast with Time-MoE Model
train_predict_teacher = []
val_predict_teacher = []
for i in range(len(tensors_train)):
    output = model.generate(tensors_train[i], max_new_tokens=1)  # shape is [batch_size, 12 + 6]
    normed_predictions = output[:, -1:]  
    train_predict_teacher.append(normed_predictions)

for i in range(len(tensors_val)):
    output = model.generate(tensors_val[i], max_new_tokens=1)  # shape is [batch_size, 12 + 6]
    normed_predictions = output[:, -1:]  
    val_predict_teacher.append(normed_predictions)

C:\Users\criju\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
# Convert the predictions to numpy arrays
r_t_train = np.vstack([t.cpu().numpy().reshape(-1,1) for t in train_predict_teacher])
r_t_val   = np.vstack([t.cpu().numpy().reshape(-1,1) for t in val_predict_teacher])

In [28]:
t_train = np.array(y_train).reshape(-1,1)
t_val   = np.array(y_val).reshape(-1,1)

In [36]:
# --- 3) Calcula σ y ε según el paper (ec.3) usando MAD sobre (t - r_t_train) ---
import numpy as np

# 1) calcula mad y sigma como antes
xi    = t_train - r_t_train
mad   = np.median(np.abs(xi - np.median(xi)))
sigma = 1.4826 * mad

# 2) elige tu alpha
alpha = 1.0

# 3) evita nan clippeando el ratio a <1
ratio = alpha / (np.sqrt(2*np.pi) * sigma)
ratio = min(ratio, 1 - 1e-6)

# 4) calcula epsilon sin miedo a nan
epsilon = sigma * np.sqrt(-2.0 * np.log(ratio))

print(f"σ = {sigma:.4f}, ratio = {ratio:.4f}, ε = {epsilon:.4f}")


σ = 0.1655, ratio = 1.0000, ε = 0.0002


In [37]:
# --- 4) Construye el student model de nuevo (dos salidas) ---
window_size, n_features, lstm_units = 15, 1, 50
inp = Input(shape=(window_size, n_features))
x   = LSTM(lstm_units)(inp)
out_clean   = Dense(1, name="clean_output")(x)   # Rs
out_teacher = Dense(1, name="teacher_output")(x) # Rd
student = Model(inp, [out_clean, out_teacher])

In [38]:
# --- 5) Define la pérdida TOR+distill ---
class TORDistillLoss(Loss):
    def __init__(self, epsilon, c_tor=1.0, c_dist=1.0, name="TORDistill"):
        super().__init__(name=name)
        self.epsilon = epsilon
        self.c_tor    = c_tor
        self.c_dist   = c_dist

    def call(self, y_true, y_pred):
        # y_true: [t, r_t] concatenados
        t      = y_true[:, 0:1]
        r_t_gt = y_true[:, 1:2]
        Rs, Rd = y_pred  # clean_output, teacher_output

        # TOR loss (ec.2)
        err = tf.abs(t - r_t_gt)
        clean_loss = tf.square(Rs - t)
        outlier_loss = tf.sqrt(tf.square(Rs - r_t_gt) + 1e-6)
        L_tor = tf.where(err < self.epsilon, clean_loss, outlier_loss)

        # Distillation loss: L1 entre Rd y r_t_gt
        L_dist = tf.abs(Rd - r_t_gt)

        return ( self.c_tor  * tf.reduce_mean(L_tor)
               + self.c_dist * tf.reduce_mean(L_dist) )

In [39]:
# --- 6) Prepara y_true concatenado para train y validation ---
y_true_train = np.concatenate([t_train, r_t_train], axis=1)
y_true_val   = np.concatenate([t_val,   r_t_val],   axis=1)

In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

class StudentTOR(Model):
    def __init__(self, window_size, n_features, lstm_units, epsilon, c_tor=1.0, c_dist=1.0):
        super().__init__()
        self.epsilon = epsilon
        self.c_tor   = c_tor
        self.c_dist  = c_dist

        self.lstm = LSTM(lstm_units, input_shape=(window_size, n_features))
        self.dense_clean   = Dense(1, name="clean_output")
        self.dense_teacher = Dense(1, name="teacher_output")

    def call(self, inputs, training=False):
        x = self.lstm(inputs)
        return self.dense_clean(x), self.dense_teacher(x)

    def compute_tor_loss(self, t, r_t_gt, Rs):
        err = tf.abs(t - r_t_gt)
        clean_loss   = tf.square(Rs - t)
        outlier_loss = tf.sqrt(tf.square(Rs - r_t_gt) + 1e-6)
        return tf.where(err < self.epsilon, clean_loss, outlier_loss)

    def train_step(self, data):
        # data = (x_batch, y_batch), y_batch shape (batch,2) con [t, r_t]
        x, y = data
        t      = y[:, 0:1]   # etiqueta real
        r_t_gt = y[:, 1:2]   # pred teacher

        with tf.GradientTape() as tape:
            Rs, Rd = self(x, training=True)     # salidas limpia y distill
            
            # 1) TOR loss
            L_tor = self.compute_tor_loss(t, r_t_gt, Rs)
            L_tor = tf.reduce_mean(L_tor)

            # 2) Distill loss (L1)
            L_dist = tf.reduce_mean(tf.abs(Rd - r_t_gt))

            loss = self.c_tor * L_tor + self.c_dist * L_dist

        # Aplica gradientes
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        # Actualiza métricas (opcional)
        self.compiled_metrics.update_state(t, Rs)  # por ej. una métrica mse sobre Rs

        return {
            "loss": loss,
            "tor_loss": L_tor,
            "distill_loss": L_dist,
            **{m.name: m.result() for m in self.metrics}
        }

    def test_step(self, data):
        # Mismo proceso para validación
        x, y = data
        t      = y[:, 0:1]
        r_t_gt = y[:, 1:2]
        Rs, Rd = self(x, training=False)

        L_tor  = tf.reduce_mean(self.compute_tor_loss(t, r_t_gt, Rs))
        L_dist = tf.reduce_mean(tf.abs(Rd - r_t_gt))
        loss   = self.c_tor * L_tor + self.c_dist * L_dist

        self.compiled_metrics.update_state(t, Rs)
        return {
            "loss": loss,
            "tor_loss": L_tor,
            "distill_loss": L_dist,
            **{m.name: m.result() for m in self.metrics}
        }


In [43]:
# 1) Calcula epsilon como antes
#    (usando MAD sobre t_train - r_t_train)
# 2) Prepara y_true concatenado:
y_true_train = np.concatenate([t_train, r_t_train], axis=1)
y_true_val   = np.concatenate([t_val,   r_t_val],   axis=1)

# 3) Instantiate and compile
student = StudentTOR(
    window_size=15,
    n_features=1,
    lstm_units=50,
    epsilon=epsilon,
    c_tor=1.0,
    c_dist=1.0
)
student.compile(optimizer='adam', metrics=[tf.keras.metrics.MeanSquaredError(name="mse")])

# 4) Fit
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = student.fit(
    X_train, y_true_train,
    validation_data=(X_val, y_true_val),
    epochs=20,
    batch_size=32,
    callbacks=[es]
)


Epoch 1/20


c:\Users\criju\.conda\envs\ts\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\criju\.conda\envs\ts\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:640: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


325/325 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - mse: 0.5795 - distill_loss: 0.2123 - loss: -0.0439 - tor_loss: 0.2303 - val_distill_loss: 0.0856 - val_loss: -0.0605 - val_tor_loss: 0.1005
Epoch 2/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - mse: 0.2984 - distill_loss: 0.1211 - loss: -0.0239 - tor_loss: 0.1215 - val_distill_loss: 0.0814 - val_loss: -0.0529 - val_tor_loss: 0.0841
Epoch 3/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - mse: 0.2787 - distill_loss: 0.0933 - loss: -0.0168 - tor_loss: 0.0942 - val_distill_loss: 0.0779 - val_loss: -0.0353 - val_tor_loss: 0.0807
Epoch 4/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - mse: 0.2489 - distill_loss: 0.0774 - loss: -0.0201 - tor_loss: 0.0784 - val_distill_loss: 0.0653 - val_loss: -0.0459 - val_tor_loss: 0.0708
Epoch 5/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - mse: 0.2590 - distill_loss: 0.0720 - loss: -0.0296 - tor_loss: 0.0722 - val_distill_loss: 0.0589 - val_loss: -0.0301 - val_tor_loss: 0.0618
Epoch 6/20
325/325 ━━━━━━━━━━━━━━━━━━━━ 1